<a href="https://colab.research.google.com/github/GiovanniTRA/text-to-taste/blob/main/get_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
import re
import json
import locale
import base64
import requests
from pprint import pprint
locale.getpreferredencoding = lambda: "UTF-8"

# OpenAI API Key
API_KEY = "<Insert your API key>"

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive/')
general_path = "/content/drive/My Drive/AriaProject_Text-to-Taste"

Mounted at /content/drive/


# 1. Perform OCR over the menu image

In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Path to your image
image_name = "PolishMenu"
image_path = f"{general_path}/recordings/{image_name}.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [ ]:
# !pip install easyocr

In [ ]:
# import easyocr
# from PIL import Image
# import numpy as np

# imsize = 3072
# confidence_thres = 0.2

# ocr_dict = {
#     'timestamps': [],
#     'texts': [],
#     'bboxes': [],
# }

# reader = easyocr.Reader(['en',])  # Load EasyOCR model. Only need to be called once.

# for sample in tqdm(sample_timestamps):

#     # Fetch image
#     image_tuple = provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
#     image_array = image_tuple[0].to_numpy_array()
#     image = Image.fromarray(image_array)
#     new_size = (imsize, imsize)
#     image = np.asarray(image.resize(new_size).rotate(-90))
#     result = reader.readtext(image)
#     print(f"result: {result}")

#     if result is not None:
#       ocr_dict['timestamps'].append(sample)
#       ocr_dict['bboxes'].append([res[0] for res in result if res[2]> confidence_thres])
#       ocr_dict['texts'].append([res[1] for res in result if res[2]> confidence_thres])


In [ ]:
def get_menu_content(base64_image):
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
  }

  payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "I have a vision impairment, can you help me read this menu"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 4096
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  return response
  menu_content = response.json()['choices'][0]['message']['content']
  return menu_content

In [ ]:
menu_content = get_menu_content(base64_image)
menu_content = menu_content.json()['choices'][0]['message']['content']

In [ ]:
f = open(f"{general_path}/ocr_polish.txt", "w")
f.write(menu_content)
f.close()

In [ ]:
pprint(menu_content)

("Of course, I'd be glad to assist. It appears this photo is depicting a menu "
 'written in Polish. Due to the curvature of the paper and the distance from '
 'the camera, some of the words may not be perfectly clear, but I will do my '
 'best to transcribe and translate what is visible:\n'
 '\n'
 '**KARTA DAŃ**\n'
 '(This means "Menu")\n'
 '\n'
 '**Przystawki** (Appetizers)\n'
 '- Przegrzebki [32,-] (Scallops)\n'
 '- Tatar z polędwicy wołowej [?] (Beef tenderloin tartare)\n'
 '- Carpaccio jagnięce [29,-] (Lamb carpaccio)\n'
 '\n'
 '**Zupy** (Soups)\n'
 '- Rosół [12,-] (Broth)\n'
 '- Chłodnik [?] (A type of cold beet soup)\n'
 '- Krem z cukinii [12,-] (Cream of zucchini)\n'
 '\n'
 '**Ryby i owoce morza** (Fish and Seafood)\n'
 '- Dorada [42,-] (Gilt-head bream)\n'
 '- Łosoś [?] (Salmon)\n'
 '- Krewetki [?] (Shrimp)\n'
 '\n'
 '**Sałatki** (Salads)\n'
 '- z krewetkami [24,-] (with shrimp)\n'
 '- Cezar [?] (Caesar)\n'
 '\n'
 '**Pasta**\n'
 '- Fregola [28,-] (a type of pasta, possibly ser

## Import food preferences

In [ ]:
with open(f"{general_path}/ocr_indian.txt", "rb") as image_file:
  menu_content = image_file.read().decode('utf-8')

In [ ]:
with open(f"{general_path}/photos.json", 'r') as f:
  photos = json.load(f)

# List of attributes to keep
attributes_to_keep = ["start_time", "address", "details"]

photos_less_info = []
for p in photos:
  new_p = {key: p[key] for key in attributes_to_keep }
  photos_less_info.append(new_p)

photos_str = json.dumps(photos_less_info, separators=(',', ':'))

In [ ]:
with open(f"{general_path}/Valia's preferences.txt", "rb") as image_file:
  preferences = image_file.read().decode('utf-8')

# 2. Get recommendations

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = API_KEY

In [ ]:
def get_recommendation(menu_content, photos_str, category_str):
  chat_model = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.1)
  template = f"""Given a menù, personal past food experiences in a JSON form and categories of food I like, you have to suggest what I should choose from the menù based on my preferences. Give me 3 recommendations"""
  human_template = """Menù: {menu}
  Personal past food experiences in JSON format: {data}
  Food categories I like: {category}"""

  chat_prompt = ChatPromptTemplate.from_messages([
      ("system", template),
      ("human", human_template),
  ])

  chain = chat_prompt | ChatOpenAI()

  recommendation = chain.invoke({"menu": menu_content, "data": photos_str, "category": category_str})
  return recommendation

In [ ]:
recommendation_output = get_recommendation(menu_content, photos_str, preferences)
recommendation_output = recommendation_output.content

In [ ]:
f = open(f"{general_path}/recommendation_output_polish.txt", "w")
f.write(recommendation_output)
f.close()

In [ ]:
with open(f"{general_path}/recommendation_output_indian.txt", "rb") as image_file:
  recommendation_output = image_file.read().decode('utf-8')

In [ ]:
pprint(recommendation_output)

('Based on your preferences and the provided menu, here are three '
 'recommendations:\n'
 '\n'
 '1. Tatar z polędwicy wołowej (Beef tenderloin tartare) from the "Przystawki" '
 '(Appetizers) section. This dish matches your preference for meat and can be '
 'a unique and flavorful choice.\n'
 '\n'
 '2. Carpaccio jagnięce (Lamb carpaccio) from the "Przystawki" (Appetizers) '
 'section. If you enjoy trying different types of meats, this lamb carpaccio '
 'can be a delicious and tender option.\n'
 '\n'
 '3. Fregola from the "Pasta" section. Since you enjoy pizza, pasta can be a '
 'great choice as well. Fregola is a unique type of pasta that you might find '
 'interesting and satisfying.\n'
 '\n'
 'Please note that the prices for certain items in the menu are obscured, so '
 'it might be helpful to inquire about the prices before making a final '
 'decision. Enjoy your meal!')


# 3. Refine prompts for image generation

In [ ]:
from openai import OpenAI

def compute_prompt_for_image_generation(recommendations):
  client = OpenAI()

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are given in input three foods and you should list in output the correspoding image generation prompt. Don't write anything else besides the enumeration list (from 1 to 3). Like '1. Fist prompt...\n2. Second prompt... \n3. Thrid prompt..."},
      {"role": "user", "content": f"Food recommendations: {recommendations}"}
    ]
  )

  food_prompts = completion.choices[0].message.content
  return food_prompts

In [ ]:
food_prompts = compute_prompt_for_image_generation(recommendation_output)

In [ ]:
f = open(f"{general_path}/food_prompts_polish.txt", "w")
f.write(food_prompts)
f.close()

In [ ]:
with open(f"{general_path}/food_prompts_indian.txt", "rb") as image_file:
  food_prompts = image_file.read().decode('utf-8')

In [ ]:
pprint(food_prompts)

('1. Generate an image of beef tenderloin tartare: \n'
 '"Please generate an image of a visually appealing beef tenderloin tartare '
 'dish with fresh ingredients and a beautiful presentation."\n'
 '\n'
 '2. Generate an image of lamb carpaccio: \n'
 '"Please generate an image of a visually stunning lamb carpaccio dish with '
 'thinly sliced lamb, garnished with herbs and spices, and served with a '
 'complementary sauce."\n'
 '\n'
 '3. Generate an image of fregola pasta: \n'
 '"Please generate an image of a mouthwatering plate of fregola pasta, cooked '
 'al dente, tossed in a flavorful sauce, and topped with vibrant and colorful '
 'ingredients."')


In [ ]:
def remove_enumeration(input_string):
    # Define a regular expression pattern to match enumeration
    pattern = re.compile(r'\d+\.\s+')

    # Use the sub method to replace matched patterns with an empty string
    result_string = pattern.sub("", input_string)

    return result_string

In [ ]:
output_string = remove_enumeration(food_prompts)

In [ ]:
pprint(output_string)

('Generate an image of beef tenderloin tartare: \n'
 '"Please generate an image of a visually appealing beef tenderloin tartare '
 'dish with fresh ingredients and a beautiful presentation."\n'
 '\n'
 'Generate an image of lamb carpaccio: \n'
 '"Please generate an image of a visually stunning lamb carpaccio dish with '
 'thinly sliced lamb, garnished with herbs and spices, and served with a '
 'complementary sauce."\n'
 '\n'
 'Generate an image of fregola pasta: \n'
 '"Please generate an image of a mouthwatering plate of fregola pasta, cooked '
 'al dente, tossed in a flavorful sauce, and topped with vibrant and colorful '
 'ingredients."')


# 4. Generate dish images

In [ ]:
def generate_image(prompt):
  client = OpenAI()

  response = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="standard",
    n=1,
  )

  image_url = response.data[0].url
  return image_url

In [ ]:
image_prompts = output_string.split("\n")
image_prompts = [t for t in image_prompts if t != ""]
dalle_outputs = []

for i in range(3):
  url = generate_image(image_prompts[i])
  dalle_outputs.append(url)

## Save images

In [ ]:
for i, url in enumerate(dalle_outputs):
  data = requests.get(url).content

  generated_folder_path = f"{general_path}/generated_photos/{image_name}"
  if not os.path.exists(generated_folder_path):
    os.makedirs(generated_folder_path)

  with open(f"{generated_folder_path}/img_{i}.jpg", 'wb') as f:
    f.write(data)

In [ ]:
import requests
from PIL import Image

def save_generated_images(image_urls, folder_path):
  for i, url in enumerate(image_urls):
    data = requests.get(url).content

    if not os.path.exists(folder_path):
      os.makedirs(folder_path)

    with open(f"{folder_path}/img_{i}.jpg", 'wb') as f:
      f.write(data)

In [ ]:
img = Image.open(f'{general_path}/img.jpg')
img.show()

# Interaction with Gradio

In [ ]:
!pip install gradio

In [ ]:
def from_menu_to_recommendations(image_path, photos_descriptions, food_categories):
  base64_image = encode_image(image_path)
  menu_content = get_menu_content(base64_image).json()['choices'][0]['message']['content']
  recommendation_output = get_recommendation(menu_content, photos_descriptions,food_categories).content
  food_prompts = compute_prompt_for_image_generation(recommendation_output)

  output_string = remove_enumeration(food_prompts)
  print(output_string)

  image_prompts = output_string.split("\n")
  image_prompts = [t for t in image_prompts if t != ""]
  dalle_outputs = []

  for i in range(3):
    url = generate_image(image_prompts[i])
    dalle_outputs.append(url)

  generated_folder_path = f"{general_path}/generated_photos/{image_name}"
  save_generated_images(dalle_outputs, generated_folder_path)

  return recommendation_output, dalle_outputs

In [ ]:
# image_path = f"{general_path}/recordings/menu.png"

def start_app(text):
  recommendation_output, generated_images = from_menu_to_recommendations(image_path, photos_str)
  # return recommendation_output, dalle_outputs[0],dalle_outputs[1], dalle_outputs[2]
  return recommendation_output, generated_images[0], generated_images[1], generated_images[2]

In [ ]:
import gradio as gr

demo = gr.Interface(fn=start_app, inputs="text", outputs=["text", "image", "image", "image"])

In [ ]:
demo.launch()

In [ ]:
# if __name__ == "__main__":
#     demo.launch(show_api=False, debug=True)